In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau

Here i try some different model architectures. Due to training taking so long the testing was not as elaborate as i would have liked, however i do think my model testing was decent since the documentation on what model architecture typically work well for multi class image classification is reasonably established.

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'tomatoe_small',
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)
class_names = dataset.class_names
n_classes = len(class_names)

Found 4000 files belonging to 10 classes.


In [3]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3


dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'TomatoeVillage',
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)
class_names = dataset.class_names
n_classes = len(class_names)

Found 16011 files belonging to 10 classes.


In [3]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

# resize_and_rescale = tf.keras.Sequential([
#   tf.keras.layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
#   tf.keras.layers.Rescaling(1./255),
# ])

rescale = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
])

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(-0.2),
  tf.keras.layers.RandomContrast(0.2),
  tf.keras.layers.RandomBrightness(0.2)
])


train_ds = train_ds.map(
    lambda x, y: (rescale(x, training=True), y)
)

val_ds = val_ds.map(
    lambda x, y: (rescale(x, training=True), y)
)

test_ds = test_ds.map(
    lambda x, y: (rescale(x, training=True), y)
)

In [8]:

#? i dont know why this function crashes the notebook
# def model_builder():
#       inputs = keras.Input(shape=(256, 256, 3), name='Input')
#       x = layers.Conv2D(filters=32, kernel_size=3, activation="relu", name='conv_layer1')(inputs)
#       x = layers.MaxPooling2D(pool_size=2 , name='pooling1')(x)
#       x = layers.Conv2D(filters=64, kernel_size=3, activation="relu" , name='conv_layer2')(x)
#       x = layers.MaxPooling2D(pool_size=2, name='pooling2')(x)
#       x = layers.Conv2D(filters=128, kernel_size=3, activation="relu", name='conv_layer3')(x)
#       x = layers.MaxPooling2D(pool_size=2, name='pooling3')(x)
#       x = layers.Conv2D(filters=256, kernel_size=3, activation="relu", name='conv_layer4')(x)
#       x = layers.MaxPooling2D(pool_size=2, name='pooling4')(x)
#       x = layers.Dropout(0.25)(x)
#       x = layers.Dense(128, activation='relu')(x)
#       x = layers.Dropout(0.25)(x)

#       outputs = layers.Dense(10, activation="sigmoid" , name='output')(x)

#       model = keras.Model(inputs=inputs, outputs=outputs, name='CNN_with_augmentation')

#       model.compile(optimizer='rmsprop', loss='SparseCategoricalCrossentropy', metrics=['accuracy'])  

#       model = model_builder()

#       history = model.fit(train_ds, validation_data=val_ds, 
#             epochs=1 , callbacks=callback_list)

#       model_metrics = pd.DataFrame(history.history)
      
#       model_scores = model.evaluate(test_ds)
#       print(f"Loss = {model_scores[0]}")
#       acc = model_scores[1]
#       formatted_acc = "{:.1%}".format(acc)
#       print(f"Accuracy = {formatted_acc}")
      
      
#       plt.clf()
#       model_scores[['accuracy','val_accuracy']].plot()

#       plt.text(0.5, 0.5, f'{formatted_acc} Accuracy on test_ds', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes, bbox=dict(facecolor='red', alpha=0.3))
#       plt.savefig('C:\\Users\\Magnus\\Desktop\\code\\timeSeries\\model_plots\\model_Q1.png')
#       plt.show()
      
#       model_metrics.tail()
      
      
#       return model_metrics, formatted_acc, 'C:\\Users\\Magnus\\Desktop\\code\\timeSeries\\model_plots\\model_Q111.png'

The pattern of increasing the filter size with eaach Conv2D generally performs well. Adding some dropout layers (or batch normalization) is common to combat overfitting. Then the model finishes with some dense layers that finally make a prediction with the output dense layer.

The literature is recommending that you dont use SparseCategoricalCrossentropy for multiclass classification, or more accurate that you one-hot encode your labels. However i saw very few do this (even experts), so it seems that in the context of neural networks, the potential negative effects of this (that the model falsely learns an ordering pattern from the label orderings) is not really an issue.

Important! I mistakenly used the sigmoid activation function in my final output layer. I should have used the softmax function. As I understand it. This makes not difference for the model performance and only helps in integrability (since scaling the predictions from 0 to 1 makes alot more sense). I correct this mistake in the final model.  

In [9]:
model = keras.Sequential([
    layers.Input(shape=(256, 256, 3), name='Input'),
    layers.Conv2D(filters=32, kernel_size=3, activation="relu", name='conv_layer1'),
    layers.MaxPooling2D(pool_size=2 , name='pooling1'),
    layers.Conv2D(filters=64, kernel_size=3, activation="relu" , name='conv_layer2'),
    layers.MaxPooling2D(pool_size=2, name='pooling2'),
    layers.Conv2D(filters=128, kernel_size=3, activation="relu", name='conv_layer3'),
    layers.MaxPooling2D(pool_size=2, name='pooling3'),
    layers.Conv2D(filters=256, kernel_size=3, activation="relu", name='conv_layer4'),
    layers.MaxPooling2D(pool_size=2, name='pooling4'),
    layers.Conv2D(filters=256, kernel_size=3, activation="relu", name='conv_layer5'),
    layers.MaxPooling2D(pool_size=2, name='pooling5'),
    layers.Dropout(0.20),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.20),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.20),
    layers.Dense(10, activation="sigmoid" , name='output')
], name='my_model')

model.compile(optimizer='rmsprop', loss='SparseCategoricalCrossentropy', metrics=['accuracy'])  

history = model.fit(train_ds, validation_data=val_ds, epochs=15)

#! 99% val acc (on the full dataset)

Epoch 1/15
100/100 [==============================] - 167s 2s/step - loss: 2.3766 - accuracy: 0.1422 - val_loss: 1.8979 - val_accuracy: 0.3750
Epoch 2/15
100/100 [==============================] - 164s 2s/step - loss: 1.8684 - accuracy: 0.3597 - val_loss: 1.4612 - val_accuracy: 0.5104
Epoch 3/15
100/100 [==============================] - 167s 2s/step - loss: 1.4209 - accuracy: 0.5163 - val_loss: 0.8624 - val_accuracy: 0.7214
Epoch 4/15
100/100 [==============================] - 168s 2s/step - loss: 1.0615 - accuracy: 0.6394 - val_loss: 1.1656 - val_accuracy: 0.5729
Epoch 5/15
100/100 [==============================] - 166s 2s/step - loss: 0.8976 - accuracy: 0.7069 - val_loss: 0.5234 - val_accuracy: 0.8203
Epoch 6/15
100/100 [==============================] - 167s 2s/step - loss: 0.7720 - accuracy: 0.7400 - val_loss: 0.6252 - val_accuracy: 0.7786
Epoch 7/15
100/100 [==============================] - 171s 2s/step - loss: 0.6381 - accuracy: 0.7834 - val_loss: 0.3083 - val_accuracy: 0.9062

In [10]:
# Save the model
model_name = 'CNV256'
os.makedirs(f'my_models/{model_name}', exist_ok=True)
model_version = '1'
model.save('my_models/{}/model_version_{}'.format(model_name,model_version))

INFO:tensorflow:Assets written to: CNV256\assets


INFO:tensorflow:Assets written to: CNV256\assets


I try using a GlobalAveragePooling2D layer , but it did not get nearly as good resutls. As I understand it, the GAP layer should have a similar effect as the dense layer. I think i implemented it wrong...

In [8]:
inputs = keras.Input(shape=(256, 256, 3), name='Input')
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu", name='conv_layer1')(inputs)
x = layers.MaxPooling2D(pool_size=2 , name='pooling1')(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu" , name='conv_layer2')(x)
x = layers.MaxPooling2D(pool_size=2, name='pooling2')(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu", name='conv_layer3')(x)
x = layers.MaxPooling2D(pool_size=2, name='pooling3')(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu", name='conv_layer4')(x)
x = layers.MaxPooling2D(pool_size=2, name='pooling4')(x)
x = layers.Conv2D(filters=512, kernel_size=3, activation="relu", name='conv_layer5')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.20)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.15)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(10, activation='softmax')(x)


model = keras.Model(inputs=inputs, outputs=outputs, name='CNN_GAP')

model.compile(optimizer='rmsprop', loss='SparseCategoricalCrossentropy', metrics=['accuracy'])  

history = model.fit(train_ds, validation_data=val_ds, 
    epochs=15)


Epoch 1/15
100/100 [==============================] - 203s 2s/step - loss: 2.2873 - accuracy: 0.1169 - val_loss: 2.3103 - val_accuracy: 0.1172
Epoch 2/15
100/100 [==============================] - 204s 2s/step - loss: 2.0999 - accuracy: 0.1997 - val_loss: 1.9498 - val_accuracy: 0.2552
Epoch 3/15
100/100 [==============================] - 195s 2s/step - loss: 1.9821 - accuracy: 0.2653 - val_loss: 1.8067 - val_accuracy: 0.3724
Epoch 4/15
100/100 [==============================] - 183s 2s/step - loss: 1.8621 - accuracy: 0.3119 - val_loss: 2.0357 - val_accuracy: 0.2135
Epoch 5/15
100/100 [==============================] - 281s 3s/step - loss: 1.8004 - accuracy: 0.3366 - val_loss: 1.6237 - val_accuracy: 0.3880
Epoch 6/15
100/100 [==============================] - 243s 2s/step - loss: 1.7513 - accuracy: 0.3622 - val_loss: 1.4848 - val_accuracy: 0.4896
Epoch 7/15
100/100 [==============================] - 254s 3s/step - loss: 1.6614 - accuracy: 0.3791 - val_loss: 1.6359 - val_accuracy: 0.4010

In [9]:
# Save the model
model_name = 'CNV512_GAP'
os.makedirs(f'my_models/{model_name}', exist_ok=True)
model_version = '1'
model.save('my_models/{}/model_version_{}'.format(model_name,model_version))

INFO:tensorflow:Assets written to: CNV512_GAP\assets


INFO:tensorflow:Assets written to: CNV512_GAP\assets


I try using a different activation function. SELU is a self-normalizing activation function that sounds very effective. I have not seen it used much at all online and considerng this model performed worse i think i understand why :/ 

I also introduced a ReduceLROnPlateau which is a way to further manipulate the learning rate. As I understand it, the rmsprop optimizer already does it to some degree, but you can use the ReduceLROnPlateau to get more control. In my case, i think the ReduceLROnPlateau would only be necessary if i trained for lots of more epochs. Kind of jojoing the learning rate up and down can help the model naviate out of a local minima to reach the global minima.

In [16]:
modelcheckpoint  = ModelCheckpoint(filepath="selu_model.ts",save_best_only=True, monitor="val_loss")

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.0001)

callback_list    = [modelcheckpoint, reduce_lr]


model = keras.Sequential([
    layers.Input(shape=(256, 256, 3), name='Input'),
    layers.Conv2D(filters=32, kernel_size=3, activation="selu", name='conv_layer1'),
    layers.MaxPooling2D(pool_size=2, name='pooling1'),
    layers.Conv2D(filters=64, kernel_size=3, activation="selu", name='conv_layer2'),
    layers.MaxPooling2D(pool_size=2, name='pooling2'),
    layers.Conv2D(filters=128, kernel_size=3, activation="selu", name='conv_layer3'),
    layers.MaxPooling2D(pool_size=2, name='pooling3'),
    layers.Conv2D(filters=256, kernel_size=3, activation="selu", name='conv_layer4'),
    layers.MaxPooling2D(pool_size=2, name='pooling4'),
    layers.BatchNormalization(name='batching1'),
    layers.Conv2D(filters=256, kernel_size=3, activation="selu", name='conv_layer5'),
    layers.MaxPooling2D(pool_size=2, name='pooling5'),
    layers.Dropout(0.20),
    layers.Conv2D(filters=384, kernel_size=3, activation="selu", name='conv_layer6'),
    layers.MaxPooling2D(pool_size=2, name='pooling6'),
    layers.Dropout(0.20),
    layers.Flatten(),
    layers.Dense(256, activation='selu'),
    layers.Dropout(0.20),
    layers.Dense(128, activation='selu'),
    layers.Dropout(0.20),
    layers.Dense(10, activation="sigmoid", name='output')
], name='CNN_SELU')

model.compile(optimizer='rmsprop', loss='SparseCategoricalCrossentropy', metrics=['accuracy'])


history = model.fit(train_ds, validation_data=val_ds, epochs=15, callbacks=callback_list)


Epoch 1/15
100/100 [==============================] - ETA: 0s - loss: 4.6671 - accuracy: 0.2000INFO:tensorflow:Assets written to: selu_model.ts\assets


INFO:tensorflow:Assets written to: selu_model.ts\assets


100/100 [==============================] - 253s 3s/step - loss: 4.6671 - accuracy: 0.2000 - val_loss: 18.5368 - val_accuracy: 0.1068 - lr: 0.0010
Epoch 2/15
100/100 [==============================] - ETA: 0s - loss: 2.6343 - accuracy: 0.2841INFO:tensorflow:Assets written to: selu_model.ts\assets


INFO:tensorflow:Assets written to: selu_model.ts\assets


100/100 [==============================] - 259s 3s/step - loss: 2.6343 - accuracy: 0.2841 - val_loss: 6.8202 - val_accuracy: 0.1797 - lr: 0.0010
Epoch 3/15
100/100 [==============================] - ETA: 0s - loss: 2.2122 - accuracy: 0.3725INFO:tensorflow:Assets written to: selu_model.ts\assets


...

100/100 [==============================] - 269s 3s/step - loss: 1.6770 - accuracy: 0.4938 - val_loss: 1.5299 - val_accuracy: 0.4479 - lr: 0.0010
Epoch 6/15
100/100 [==============================] - 272s 3s/step - loss: 1.5748 - accuracy: 0.5297 - val_loss: 2.7216 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 7/15
100/100 [==============================] - 272s 3s/step - loss: 1.4299 - accuracy: 0.5497 - val_loss: 1.8488 - val_accuracy: 0.3984 - lr: 0.0010
Epoch 8/15
100/100 [==============================] - 277s 3s/step - loss: 1.2545 - accuracy: 0.6081 - val_loss: 3.0183 - val_accuracy: 0.3828 - lr: 0.0010
Epoch 9/15
100/100 [==============================] - 284s 3s/step - loss: 1.1765 - accuracy: 0.6269 - val_loss: 1.8724 - val_accuracy: 0.5885 - lr: 0.0010
Epoch 10/15
100/100 [==============================] - 286s 3s/step - loss: 1.1029 - accuracy: 0.6528 - val_loss: 7.6288 - val_accuracy: 0.3073 - lr: 0.0010
Epoch 11/15
100/100 [==============================] - ETA: 0s - loss: 0.7

INFO:tensorflow:Assets written to: selu_model.ts\assets


100/100 [==============================] - 231s 2s/step - loss: 0.7113 - accuracy: 0.7466 - val_loss: 0.4557 - val_accuracy: 0.8307 - lr: 1.0000e-04
Epoch 12/15
100/100 [==============================] - 210s 2s/step - loss: 0.6215 - accuracy: 0.7828 - val_loss: 0.5516 - val_accuracy: 0.7917 - lr: 1.0000e-04
Epoch 13/15
100/100 [==============================] - 232s 2s/step - loss: 0.5570 - accuracy: 0.8003 - val_loss: 0.5343 - val_accuracy: 0.8047 - lr: 1.0000e-04
Epoch 14/15
100/100 [==============================] - 214s 2s/step - loss: 0.5057 - accuracy: 0.8244 - val_loss: 0.4658 - val_accuracy: 0.8281 - lr: 1.0000e-04
Epoch 15/15
100/100 [==============================] - ETA: 0s - loss: 0.4705 - accuracy: 0.8319INFO:tensorflow:Assets written to: selu_model.ts\assets


INFO:tensorflow:Assets written to: selu_model.ts\assets


100/100 [==============================] - 199s 2s/step - loss: 0.4705 - accuracy: 0.8319 - val_loss: 0.3519 - val_accuracy: 0.8906 - lr: 1.0000e-04


In [17]:
# Save the model
model_name = 'selu_384'
os.makedirs(f'my_models/{model_name}', exist_ok=True)
model_version = '1'
model.save('my_models/{}/model_version_{}'.format(model_name,model_version))

INFO:tensorflow:Assets written to: selu_384\assets


INFO:tensorflow:Assets written to: selu_384\assets


This final model is pretty similar to the first baseline model. The only difference being some more Conv2D layers and some Dropout layers and a BatchNormalization layer. This model does learn a bit slower than the baseline model but it reached the same result with some more training epochs and i have more confidence in its performance due to extra regularization. 

I save and load the model a few times so that i can train the model more. I end up training it for 35+ epochs

In [19]:
modelcheckpoint  = ModelCheckpoint(filepath="relu_model.ts",save_best_only=True, monitor="val_loss")

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.0001)

callback_list    = [modelcheckpoint, reduce_lr]


model = keras.Sequential([
    layers.Input(shape=(256, 256, 3), name='Input'),
    layers.Conv2D(filters=32, kernel_size=3, activation="relu", name='conv_layer1'),
    layers.MaxPooling2D(pool_size=2, name='pooling1'),
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", name='conv_layer2'),
    layers.MaxPooling2D(pool_size=2, name='pooling2'),
    layers.Conv2D(filters=128, kernel_size=3, activation="relu", name='conv_layer3'),
    layers.MaxPooling2D(pool_size=2, name='pooling3'),
    layers.Conv2D(filters=256, kernel_size=3, activation="relu", name='conv_layer4'),
    layers.MaxPooling2D(pool_size=2, name='pooling4'),
    layers.BatchNormalization(name='batching1'),
    layers.Conv2D(filters=256, kernel_size=3, activation="relu", name='conv_layer5'),
    layers.MaxPooling2D(pool_size=2, name='pooling5'),
    layers.Dropout(0.20),
    layers.Conv2D(filters=384, kernel_size=3, activation="relu", name='conv_layer6'),
    layers.MaxPooling2D(pool_size=2, name='pooling6'),
    layers.Dropout(0.20),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.20),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.20),
    layers.Dense(10, activation="sigmoid", name='output')
], name='CNN_relu')

model.compile(optimizer='rmsprop', loss='SparseCategoricalCrossentropy', metrics=['accuracy'])


history = model.fit(train_ds, validation_data=val_ds, epochs=15, callbacks=callback_list)

Epoch 1/15
100/100 [==============================] - ETA: 0s - loss: 2.1846 - accuracy: 0.2025INFO:tensorflow:Assets written to: relu_model.ts\assets


INFO:tensorflow:Assets written to: relu_model.ts\assets


100/100 [==============================] - 190s 2s/step - loss: 2.1846 - accuracy: 0.2025 - val_loss: 2.2639 - val_accuracy: 0.1432 - lr: 0.0010
Epoch 2/15
100/100 [==============================] - ETA: 0s - loss: 1.7747 - accuracy: 0.3609INFO:tensorflow:Assets written to: relu_model.ts\assets


INFO:tensorflow:Assets written to: relu_model.ts\assets


100/100 [==============================] - 197s 2s/step - loss: 1.7747 - accuracy: 0.3609 - val_loss: 2.2316 - val_accuracy: 0.1875 - lr: 0.0010
Epoch 3/15
100/100 [==============================] - ETA: 0s - loss: 1.4259 - accuracy: 0.5072INFO:tensorflow:Assets written to: relu_model.ts\assets


...

100/100 [==============================] - 199s 2s/step - loss: 0.7057 - accuracy: 0.7697 - val_loss: 0.7648 - val_accuracy: 0.7109 - lr: 0.0010
Epoch 8/15
100/100 [==============================] - 186s 2s/step - loss: 0.6695 - accuracy: 0.7809 - val_loss: 1.3731 - val_accuracy: 0.5703 - lr: 0.0010
Epoch 9/15
100/100 [==============================] - ETA: 0s - loss: 0.5990 - accuracy: 0.8012INFO:tensorflow:Assets written to: relu_model.ts\assets


INFO:tensorflow:Assets written to: relu_model.ts\assets


100/100 [==============================] - 182s 2s/step - loss: 0.5990 - accuracy: 0.8012 - val_loss: 0.5180 - val_accuracy: 0.8073 - lr: 0.0010
Epoch 10/15
100/100 [==============================] - 191s 2s/step - loss: 0.5077 - accuracy: 0.8334 - val_loss: 0.7356 - val_accuracy: 0.7448 - lr: 0.0010
Epoch 11/15
100/100 [==============================] - 194s 2s/step - loss: 0.4874 - accuracy: 0.8388 - val_loss: 0.6025 - val_accuracy: 0.7682 - lr: 0.0010
Epoch 12/15
100/100 [==============================] - 189s 2s/step - loss: 0.4490 - accuracy: 0.8500 - val_loss: 1.0424 - val_accuracy: 0.7031 - lr: 0.0010
Epoch 13/15
100/100 [==============================] - 195s 2s/step - loss: 0.3716 - accuracy: 0.8772 - val_loss: 1.7620 - val_accuracy: 0.6328 - lr: 0.0010
Epoch 14/15
100/100 [==============================] - 209s 2s/step - loss: 0.3982 - accuracy: 0.8766 - val_loss: 1.3847 - val_accuracy: 0.6719 - lr: 0.0010
Epoch 15/15
100/100 [==============================] - ETA: 0s - loss:

INFO:tensorflow:Assets written to: relu_model.ts\assets


100/100 [==============================] - 192s 2s/step - loss: 0.2143 - accuracy: 0.9294 - val_loss: 0.1074 - val_accuracy: 0.9661 - lr: 1.0000e-04


In [12]:
# Save the model
model_name = 'relu_384'
os.makedirs(f'my_models/{model_name}', exist_ok=True)
model_version = '1'
model.save('my_models/{}/model_version_{}'.format(model_name,model_version))


INFO:tensorflow:Assets written to: relu_384\assets


INFO:tensorflow:Assets written to: relu_384\assets


In [11]:
# load the saved model
model = tf.keras.models.load_model('relu_384')
history = model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=callback_list)

# Save the model
model_name = 'relu_384'
os.makedirs(f'my_models/{model_name}', exist_ok=True)
model_version = '1'
model.save('my_models/{}/model_version_{}'.format(model_name,model_version))

Epoch 1/10


100/100 [==============================] - 184s 2s/step - loss: 0.0435 - accuracy: 0.9853 - val_loss: 0.0059 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 0.0354 - accuracy: 0.9897INFO:tensorflow:Assets written to: relu_model.ts\assets


INFO:tensorflow:Assets written to: relu_model.ts\assets


100/100 [==============================] - 256s 3s/step - loss: 0.0354 - accuracy: 0.9897 - val_loss: 0.0033 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 0.0283 - accuracy: 0.9912INFO:tensorflow:Assets written to: relu_model.ts\assets


INFO:tensorflow:Assets written to: relu_model.ts\assets


100/100 [==============================] - 235s 2s/step - loss: 0.0283 - accuracy: 0.9912 - val_loss: 0.0028 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 4/10
100/100 [==============================] - ETA: 0s - loss: 0.0240 - accuracy: 0.9909INFO:tensorflow:Assets written to: relu_model.ts\assets


INFO:tensorflow:Assets written to: relu_model.ts\assets


100/100 [==============================] - 252s 2s/step - loss: 0.0240 - accuracy: 0.9909 - val_loss: 0.0024 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 5/10
100/100 [==============================] - ETA: 0s - loss: 0.0274 - accuracy: 0.9925INFO:tensorflow:Assets written to: relu_model.ts\assets


INFO:tensorflow:Assets written to: relu_model.ts\assets


100/100 [==============================] - 265s 3s/step - loss: 0.0274 - accuracy: 0.9925 - val_loss: 5.0466e-04 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 6/10
100/100 [==============================] - 248s 2s/step - loss: 0.0257 - accuracy: 0.9916 - val_loss: 0.0100 - val_accuracy: 0.9974 - lr: 1.0000e-04
Epoch 7/10
100/100 [==============================] - 254s 3s/step - loss: 0.0251 - accuracy: 0.9916 - val_loss: 0.0015 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 8/10
100/100 [==============================] - 271s 3s/step - loss: 0.0339 - accuracy: 0.9894 - val_loss: 6.9156e-04 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 9/10
100/100 [==============================] - 268s 3s/step - loss: 0.0390 - accuracy: 0.9856 - val_loss: 0.0031 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 10/10
100/100 [==============================] - 425s 4s/step - loss: 0.0281 - accuracy: 0.9909 - val_loss: 5.5441e-04 - val_accuracy: 1.0000 - lr: 1.0000e-04
